In [1]:
pic_='F'
tracking_uri_="http://34.128.104.38:5000"

In [2]:
# Parameters
pic_ = "F"
tracking_uri_ = "http://34.128.104.38:5000"


In [3]:
import pandas as pd 
import numpy as np
import os
import scipy
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")


from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score,log_loss

from sklearn.ensemble import RandomForestClassifier

import pickle5 as pickle
import os

from google.cloud import storage
client = storage.Client()
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'gcp_store.json'

from multiprocessing import Pool as pool_x

from itertools import repeat
import mlflow
from mlflow.tracking import MlflowClient


In [4]:
def parallel_FE_rforest(i):
    global pic_,status_, train_x, valid_x, train_y, valid_y, columns_data, strategy_
    
    experiment_name = "Feature Selection"
    ## check if the experiment already exists
    if not mlflow.get_experiment_by_name(experiment_name):
        mlflow.create_experiment(name=experiment_name) 
    experiment = mlflow.get_experiment_by_name(experiment_name)
    ### check finished experiment
    client = MlflowClient()
    exp_r = client.get_experiment_by_name("Feature Selection")
    client_run=mlflow.search_runs([exp_r.experiment_id], order_by=["metrics.f1_micro DESC"])
    if ((client_run.shape[0]>0)&("params.condition" in client_run.columns)&
        ("params.algorithm" in client_run.columns)&("params.strategy" in client_run.columns)):
        client_run=client_run.loc[(client_run["params.condition"]==status_)&
                                  (client_run["params.algorithm"]=='randomforest')&
                                  (client_run["params.strategy"]==strategy_)]
        if str(i) in client_run['params.num_feature'].to_list():
            print('Experiment with number of feature:',i, ' already done. It will be skipped.')
            return ''
    mlflow.set_tracking_uri(tracking_uri_)
    
    with mlflow.start_run(experiment_id = experiment.experiment_id,
                          run_name=status_+'-'+datetime.now().strftime("%d/%m/%Y %H:%M:%S")):
        model=RandomForestClassifier(n_estimators=20,random_state=1234)
        selector = RFE(model, n_features_to_select=i, step=1)
        selector.fit(train_x, train_y)
        
        pred_labels=selector.predict(valid_x).tolist()
        preds_proba_=selector.predict_proba(valid_x)
        
        
        f1_micro=f1_score(valid_y, pred_labels,average='micro')
        log_loss_=log_loss(valid_y,preds_proba_)
        col_data=[columns_data[j] for j in list(np.where(selector.support_)[0])]    
        
        mlflow.log_param("num_feature", i)
        mlflow.log_param('feature_name',col_data)
        mlflow.log_param('condition',status_)
        mlflow.log_param('algorithm',"randomforest")        
        mlflow.log_param('strategy',strategy_)

        mlflow.log_metric("f1_micro", f1_micro)
        mlflow.log_metric("log_loss", log_loss_)
        
    return ''

In [5]:
def run_FE_rforest():
    y1=range(5,18)
    with pool_x(28) as p:
        p.map(parallel_FE_rforest,y1)
    #for i1 in y1:
    #    parallel_FE_rforest(i1)

### Without Artificial Data

In [6]:
print('RUN DATA WITHOUT ARTIFICIAL DATA\n\n')
print('===========RUN Data w Outlier===========')
strategy_='no artificial'
data_sample=pd.read_csv("gs://bps-gcp-bucket/MLST2023/preprocessing/sample_"+str(pic_) +".csv",sep=',')
data_sample['nama_valid']=data_sample.nama_valid.apply(lambda y: str(y)[:6])
data_sample=data_sample.sort_values('nama_valid')
status_='w_outlier_'
columns_data=['B1_p15', 'B2_p15', 'B3_p15',
       'B4_p15', 'B5_p15', 'B6_p15', 'B7_p15', 'B8_p15', 'B8A_p15', 'B11_p15',
       'B12_p15', 'NDVI_p50', 'NDWI_p50', 'NDBI_p50', 'SAVI_p50', 'EVI_p50',
       'GNDVI_p50','nama_valid']
X=data_sample[columns_data].drop(['nama_valid'],axis=1)
y=data_sample['nama_valid']
train_x, valid_x, train_y, valid_y = train_test_split(X, y, test_size=0.2,stratify=y)
run_FE_rforest()
print('===========FINISH=======================')
print('===========RUN Data w/o Outlier===========')
data_sample=pd.read_csv("gs://bps-gcp-bucket/MLST2023/preprocessing/sample_"+str(pic_) +"_no_outlier.csv",sep=',')
data_sample['nama_valid']=data_sample.nama_valid.apply(lambda y: str(y)[:6])
data_sample=data_sample.sort_values('nama_valid')
status_='wo_outlier_'
columns_data=['B1_p15', 'B2_p15', 'B3_p15',
       'B4_p15', 'B5_p15', 'B6_p15', 'B7_p15', 'B8_p15', 'B8A_p15', 'B11_p15',
       'B12_p15', 'NDVI_p50', 'NDWI_p50', 'NDBI_p50', 'SAVI_p50', 'EVI_p50',
       'GNDVI_p50','nama_valid']
X=data_sample[columns_data].drop(['nama_valid'],axis=1)
y=data_sample['nama_valid']
train_x, valid_x, train_y, valid_y = train_test_split(X, y, test_size=0.2,stratify=y)
run_FE_rforest()
print('===========FINISH=======================')
print('\n\nFINISH')

RUN DATA WITHOUT ARTIFICIAL DATA


===========RUN Data w Outlier===========


Experiment with number of feature:

Experiment with number of feature:

16

5

...

 already done. It will be skipped.

 already done. It will be skipped.

17

 already done. It will be skipped.

===========FINISH=======================


FINISH


### ENN-BORDER SMOTE

In [7]:
print('RUN DATA WITHOUT ENN-BORDER SMOTE\n\n')
print('===========RUN Data w Outlier===========')
strategy_='enn_border_smote'
data_sample=pd.read_csv("gs://bps-gcp-bucket/MLST2023/preprocessing/sample_"+str(pic_) +"_enn_border_smote.csv",sep=',')
data_sample['nama_valid']=data_sample.nama_valid.apply(lambda y: str(y)[:6])
data_sample=data_sample.sort_values('nama_valid')
status_='w_outlier_'
columns_data=['B1_p15', 'B2_p15', 'B3_p15',
       'B4_p15', 'B5_p15', 'B6_p15', 'B7_p15', 'B8_p15', 'B8A_p15', 'B11_p15',
       'B12_p15', 'NDVI_p50', 'NDWI_p50', 'NDBI_p50', 'SAVI_p50', 'EVI_p50',
       'GNDVI_p50','nama_valid']
X=data_sample[columns_data].drop(['nama_valid'],axis=1)
y=data_sample['nama_valid']
train_x, valid_x, train_y, valid_y = train_test_split(X, y, test_size=0.2,stratify=y)
run_FE_rforest()
print('===========FINISH=======================')
print('===========RUN Data w/o Outlier===========')
data_sample=pd.read_csv("gs://bps-gcp-bucket/MLST2023/preprocessing/sample_"+str(pic_) +"_no_outlier_enn_border_smote.csv",sep=',')
data_sample['nama_valid']=data_sample.nama_valid.apply(lambda y: str(y)[:6])
data_sample=data_sample.sort_values('nama_valid')
status_='wo_outlier_'
columns_data=['B1_p15', 'B2_p15', 'B3_p15',
       'B4_p15', 'B5_p15', 'B6_p15', 'B7_p15', 'B8_p15', 'B8A_p15', 'B11_p15',
       'B12_p15', 'NDVI_p50', 'NDWI_p50', 'NDBI_p50', 'SAVI_p50', 'EVI_p50',
       'GNDVI_p50','nama_valid']
X=data_sample[columns_data].drop(['nama_valid'],axis=1)
y=data_sample['nama_valid']
train_x, valid_x, train_y, valid_y = train_test_split(X, y, test_size=0.2,stratify=y)
run_FE_rforest()
print('===========FINISH=======================')
print('\n\nFINISH')

RUN DATA WITHOUT ENN-BORDER SMOTE


===========RUN Data w Outlier===========


Experiment with number of feature:

14

 already done. It will be skipped.

Experiment with number of feature:

...

 already done. It will be skipped.

Experiment with number of feature:

5

 already done. It will be skipped.

===========FINISH=======================


FINISH


### ENN-SMOTE

In [8]:
print('RUN DATA WITHOUT ENN-SMOTE\n\n')
print('===========RUN Data w Outlier===========')
strategy_='enn_smote'
data_sample=pd.read_csv("gs://bps-gcp-bucket/MLST2023/preprocessing/sample_"+str(pic_) +"_enn_smote.csv",sep=',')
data_sample['nama_valid']=data_sample.nama_valid.apply(lambda y: str(y)[:6])
data_sample=data_sample.sort_values('nama_valid')
status_='w_outlier_'
columns_data=['B1_p15', 'B2_p15', 'B3_p15',
       'B4_p15', 'B5_p15', 'B6_p15', 'B7_p15', 'B8_p15', 'B8A_p15', 'B11_p15',
       'B12_p15', 'NDVI_p50', 'NDWI_p50', 'NDBI_p50', 'SAVI_p50', 'EVI_p50',
       'GNDVI_p50','nama_valid']
X=data_sample[columns_data].drop(['nama_valid'],axis=1)
y=data_sample['nama_valid']
train_x, valid_x, train_y, valid_y = train_test_split(X, y, test_size=0.2,stratify=y)
run_FE_rforest()
print('===========FINISH=======================')
print('===========RUN Data w/o Outlier===========')
data_sample=pd.read_csv("gs://bps-gcp-bucket/MLST2023/preprocessing/sample_"+str(pic_) +"_no_outlier_enn_smote.csv",sep=',')
data_sample['nama_valid']=data_sample.nama_valid.apply(lambda y: str(y)[:6])
data_sample=data_sample.sort_values('nama_valid')
status_='wo_outlier_'
columns_data=['B1_p15', 'B2_p15', 'B3_p15',
       'B4_p15', 'B5_p15', 'B6_p15', 'B7_p15', 'B8_p15', 'B8A_p15', 'B11_p15',
       'B12_p15', 'NDVI_p50', 'NDWI_p50', 'NDBI_p50', 'SAVI_p50', 'EVI_p50',
       'GNDVI_p50','nama_valid']
X=data_sample[columns_data].drop(['nama_valid'],axis=1)
y=data_sample['nama_valid']
train_x, valid_x, train_y, valid_y = train_test_split(X, y, test_size=0.2,stratify=y)
run_FE_rforest()
print('===========FINISH=======================')
print('\n\nFINISH')

RUN DATA WITHOUT ENN-SMOTE


===========RUN Data w Outlier===========


Experiment with number of feature:

Experiment with number of feature:

Experiment with number of feature:

14

...

 already done. It will be skipped.

Experiment with number of feature:

5

 already done. It will be skipped.

===========FINISH=======================


FINISH


### TL-BORDER SMOTE

In [9]:
print('RUN DATA WITHOUT TL-BORDER SMOTE\n\n')
print('===========RUN Data w Outlier===========')
strategy_='tl_border_smote'
data_sample=pd.read_csv("gs://bps-gcp-bucket/MLST2023/preprocessing/sample_"+str(pic_) +"_tl_border_smote.csv",sep=',')
data_sample['nama_valid']=data_sample.nama_valid.apply(lambda y: str(y)[:6])
data_sample=data_sample.sort_values('nama_valid')
status_='w_outlier_'
columns_data=['B1_p15', 'B2_p15', 'B3_p15',
       'B4_p15', 'B5_p15', 'B6_p15', 'B7_p15', 'B8_p15', 'B8A_p15', 'B11_p15',
       'B12_p15', 'NDVI_p50', 'NDWI_p50', 'NDBI_p50', 'SAVI_p50', 'EVI_p50',
       'GNDVI_p50','nama_valid']
X=data_sample[columns_data].drop(['nama_valid'],axis=1)
y=data_sample['nama_valid']
train_x, valid_x, train_y, valid_y = train_test_split(X, y, test_size=0.2,stratify=y)
run_FE_rforest()
print('===========FINISH=======================')
print('===========RUN Data w/o Outlier===========')
data_sample=pd.read_csv("gs://bps-gcp-bucket/MLST2023/preprocessing/sample_"+str(pic_) +"_no_outlier_tl_border_smote.csv",sep=',')
data_sample['nama_valid']=data_sample.nama_valid.apply(lambda y: str(y)[:6])
data_sample=data_sample.sort_values('nama_valid')
status_='wo_outlier_'
columns_data=['B1_p15', 'B2_p15', 'B3_p15',
       'B4_p15', 'B5_p15', 'B6_p15', 'B7_p15', 'B8_p15', 'B8A_p15', 'B11_p15',
       'B12_p15', 'NDVI_p50', 'NDWI_p50', 'NDBI_p50', 'SAVI_p50', 'EVI_p50',
       'GNDVI_p50','nama_valid']
X=data_sample[columns_data].drop(['nama_valid'],axis=1)
y=data_sample['nama_valid']
train_x, valid_x, train_y, valid_y = train_test_split(X, y, test_size=0.2,stratify=y)
run_FE_rforest()
print('===========FINISH=======================')
print('\n\nFINISH')

RUN DATA WITHOUT TL-BORDER SMOTE


===========RUN Data w Outlier===========


Experiment with number of feature:

14

 already done. It will be skipped.

Experiment with number of feature:

...

 already done. It will be skipped.

Experiment with number of feature:

6

 already done. It will be skipped.

===========FINISH=======================


FINISH


### TL-SMOTE

In [10]:
print('RUN DATA WITHOUT TL-SMOTE\n\n')
print('===========RUN Data w Outlier===========')
strategy_='tl_smote'
data_sample=pd.read_csv("gs://bps-gcp-bucket/MLST2023/preprocessing/sample_"+str(pic_) +"_tl_smote.csv",sep=',')
data_sample['nama_valid']=data_sample.nama_valid.apply(lambda y: str(y)[:6])
data_sample=data_sample.sort_values('nama_valid')
status_='w_outlier_'
columns_data=['B1_p15', 'B2_p15', 'B3_p15',
       'B4_p15', 'B5_p15', 'B6_p15', 'B7_p15', 'B8_p15', 'B8A_p15', 'B11_p15',
       'B12_p15', 'NDVI_p50', 'NDWI_p50', 'NDBI_p50', 'SAVI_p50', 'EVI_p50',
       'GNDVI_p50','nama_valid']
X=data_sample[columns_data].drop(['nama_valid'],axis=1)
y=data_sample['nama_valid']
train_x, valid_x, train_y, valid_y = train_test_split(X, y, test_size=0.2,stratify=y)
run_FE_rforest()
print('===========FINISH=======================')
print('===========RUN Data w/o Outlier===========')
data_sample=pd.read_csv("gs://bps-gcp-bucket/MLST2023/preprocessing/sample_"+str(pic_) +"_no_outlier_tl_smote.csv",sep=',')
data_sample['nama_valid']=data_sample.nama_valid.apply(lambda y: str(y)[:6])
data_sample=data_sample.sort_values('nama_valid')
status_='wo_outlier_'
columns_data=['B1_p15', 'B2_p15', 'B3_p15',
       'B4_p15', 'B5_p15', 'B6_p15', 'B7_p15', 'B8_p15', 'B8A_p15', 'B11_p15',
       'B12_p15', 'NDVI_p50', 'NDWI_p50', 'NDBI_p50', 'SAVI_p50', 'EVI_p50',
       'GNDVI_p50','nama_valid']
X=data_sample[columns_data].drop(['nama_valid'],axis=1)
y=data_sample['nama_valid']
train_x, valid_x, train_y, valid_y = train_test_split(X, y, test_size=0.2,stratify=y)
run_FE_rforest()
print('===========FINISH=======================')
print('\n\nFINISH')

RUN DATA WITHOUT TL-SMOTE


===========RUN Data w Outlier===========


Experiment with number of feature:

Experiment with number of feature:

6

8

...

13

9

 already done. It will be skipped.

 already done. It will be skipped.

===========FINISH=======================


FINISH
